# MERGE brickset WITH kaggle

**Workflow** 1<br>
**Goal** Merge kaggle data into the brickset.<br>
**MAKES** data/brickset.csv

In [1]:
import os
import pandas as pd

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', -1)

In [3]:
%load_ext autoreload
%autoreload 2

import brickset_helper as bh

In [4]:
brickset = pd.read_csv("data/brickset_set_clean.csv")
brickset[["set_no", "piece_cnt"]].head(3)

,set_no,piece_cnt
0,722-1,301.0
1,733-1,533.0
2,744-1,537.0


In [5]:
## from phil

remove_group_ls = ['Basic', 'Constraction', 'Educational',  'Miscellaneous', 'Pre-school',
                  'Vintage themes']
brickset = brickset.loc[brickset['theme_group'].isin(remove_group_ls)==False]

## merge inventory_id as new column

In [6]:
kaggle_inventories = pd.read_csv("data/inventories.csv")

kaggle_inventories.head(3)

,id,version,set_num
0,1,1,7922-1
1,3,1,3931-1
2,4,1,6942-1


In [7]:
## select needed columns
kaggle_inventories = kaggle_inventories[["set_num", "id"]]

## rename columns for merging
kaggle_inventories.columns = ["set_no", "inventory_id"]

kaggle_inventories.head(3)

,set_no,inventory_id
0,7922-1,1
1,3931-1,3
2,6942-1,4


In [8]:
brickset = pd.merge(brickset, kaggle_inventories, how="left")

## count missing entries
print(F'missing inventory_id for {brickset["inventory_id"].isna().sum()} entries')

brickset[["set_no", "piece_cnt", "inventory_id"]].head(3)

missing inventory_id for 1154 entries


,set_no,piece_cnt,inventory_id
0,858-1,242.0,4886.0
1,1591-1,40.0,11940.0
2,1592-1,471.0,14106.0


In [9]:
## integer columns get cast to a float if NaNs are introduced
## some sets do not have an inventory_id

## delete NANs from "inventory_id"
brickset.dropna(subset=["inventory_id"], inplace=True)

## reset "inventory_id" to integers
brickset["inventory_id"] = brickset["inventory_id"].astype("int")

brickset[["set_no", "piece_cnt", "inventory_id"]].head(3)

,set_no,piece_cnt,inventory_id
0,858-1,242.0,4886
1,1591-1,40.0,11940
2,1592-1,471.0,14106


## process inventory_parts + colors

In [10]:
kaggle_inventory_parts = pd.read_csv("data/inventory_parts.csv")

kaggle_inventory_parts.head(3)

,inventory_id,part_num,color_id,quantity,is_spare
0,1,48379c01,72,1,f
1,1,48395,7,1,f
2,1,mcsport6,25,1,f


In [11]:
## subset columns
kaggle_inventory_parts = kaggle_inventory_parts[["inventory_id", "part_num", "color_id", "quantity"]]

kaggle_inventory_parts.head(3)

,inventory_id,part_num,color_id,quantity
0,1,48379c01,72,1
1,1,48395,7,1
2,1,mcsport6,25,1


In [12]:
kaggle_colors = pd.read_csv("data/colors.csv")

kaggle_colors.head(3)

,id,name,rgb,is_trans
0,-1,Unknown,0033B2,f
1,0,Black,05131D,f
2,1,Blue,0055BF,f


In [13]:
## subset columns
kaggle_colors = kaggle_colors[["id", "name"]]

## change column names for merge
kaggle_colors.columns = ["color_id", "color_name"]

kaggle_colors.head(3)

,color_id,color_name
0,-1,Unknown
1,0,Black
2,1,Blue


In [14]:
## merge
kaggle_parts = pd.merge(kaggle_inventory_parts, kaggle_colors, how="left")

kaggle_parts.head(3)

,inventory_id,part_num,color_id,quantity,color_name
0,1,48379c01,72,1,Dark Bluish Gray
1,1,48395,7,1,Light Gray
2,1,mcsport6,25,1,Orange


## merge number_of_parts

In [15]:
kaggle_number_parts = kaggle_parts[["inventory_id", "quantity"]].groupby(by="inventory_id").count()
kaggle_number_parts = kaggle_number_parts.reset_index()

## rename columns for merging
kaggle_number_parts.columns = ["inventory_id", "number_parts"]

kaggle_number_parts.head(3)

,inventory_id,number_parts
0,1,4
1,3,29
2,4,16


In [16]:
brickset = pd.merge(brickset, kaggle_number_parts, how="left")

## count missing entries
print(F'missing number_parts for {brickset["number_parts"].isna().sum()} entries')

brickset[["set_no", "inventory_id", "number_parts"]].head(3)

missing number_parts for 173 entries


,set_no,inventory_id,number_parts
0,858-1,4886,46.0
1,1591-1,11940,19.0
2,1592-1,14106,133.0


In [17]:
## delete NANs from "number_parts"
brickset.dropna(subset=["number_parts"], inplace=True)

## reset "number_parts" to integers
brickset["number_parts"] = brickset["number_parts"].astype("int")

brickset[["set_no", "inventory_id", "number_parts"]].head(3)

,set_no,inventory_id,number_parts
0,858-1,4886,46
1,1591-1,11940,19
2,1592-1,14106,133


## merge total_parts

In [18]:
kaggle_total_parts = kaggle_parts[["inventory_id", "quantity"]].groupby(by="inventory_id").sum()
kaggle_total_parts = kaggle_total_parts.reset_index()

## reset column names for merging
kaggle_total_parts.columns = ["inventory_id", "total_parts"]

kaggle_total_parts.head(3)

,inventory_id,total_parts
0,1,4
1,3,46
2,4,21


In [19]:
brickset = pd.merge(brickset, kaggle_total_parts, how="left")

## count missing entries
print(F'missing total_parts for {brickset["total_parts"].isna().sum()} entries')

brickset[["set_no", "inventory_id", "number_parts", "total_parts"]].head(3)

missing total_parts for 0 entries


,set_no,inventory_id,number_parts,total_parts
0,858-1,4886,46,242
1,1591-1,11940,19,40
2,1592-1,14106,133,495


## throw away sets with no repeat pieces

In [20]:
filter = brickset["number_parts"] != brickset["total_parts"]
brickset = brickset[filter]
brickset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5056 entries, 0 to 5294
Data columns (total 22 columns):
set_no           5056 non-null object
name             5056 non-null object
url              5056 non-null object
theme_group      5056 non-null object
theme            5056 non-null object
subtheme         3757 non-null object
year             5056 non-null float64
piece_cnt        5013 non-null float64
minifig_cnt      5056 non-null float64
inventory_url    3991 non-null object
minifig_url      3549 non-null object
price_store      3858 non-null float64
price_new        4722 non-null float64
price_used       4639 non-null float64
rating_value     4237 non-null float64
rating_votes     4237 non-null float64
tags             4867 non-null object
set_type         5056 non-null object
packaging        4672 non-null object
inventory_id     5056 non-null int64
number_parts     5056 non-null int64
total_parts      5056 non-null int64
dtypes: float64(8), int64(3), object(11)
memory usag

## merge colors

In [21]:
kaggle_parts.head(3)

,inventory_id,part_num,color_id,quantity,color_name
0,1,48379c01,72,1,Dark Bluish Gray
1,1,48395,7,1,Light Gray
2,1,mcsport6,25,1,Orange


In [22]:
kaggle_part_colors = kaggle_parts.groupby(["inventory_id", "color_name"])[["quantity"]].sum()
kaggle_part_colors = kaggle_part_colors.reset_index()

kaggle_part_colors = kaggle_part_colors.groupby(["inventory_id"]).apply(lambda row: row.nlargest(2, 'quantity'))

kaggle_part_colors = kaggle_part_colors.drop(columns="inventory_id")

kaggle_part_colors = kaggle_part_colors.reset_index()
kaggle_part_colors

,inventory_id,level_1,color_name,quantity
0,1,0,Black,1
1,1,1,Dark Bluish Gray,1
2,3,6,Bright Pink,10
3,3,9,Lime,10
4,4,20,Trans-Neon Green,7
...,...,...,...,...
20426,18645,94344,Black,1
20427,18704,94352,Black,118
20428,18704,94356,Dark Bluish Gray,63
20429,18708,94377,Dark Bluish Gray,58


In [23]:
brickset_primary = {}

for row in brickset.itertuples():
    filter = kaggle_part_colors["inventory_id"] == row.inventory_id
    brickset.loc[row.Index, "1st_color"] = (kaggle_part_colors[filter].iloc[0].loc["color_name"])
    brickset.loc[row.Index, "2nd_color"] = (kaggle_part_colors[filter].iloc[-1].loc["color_name"])

brickset[["set_no", "inventory_id", "number_parts", "total_parts", "1st_color", "2nd_color"]].head(3)

,set_no,inventory_id,number_parts,total_parts,1st_color,2nd_color
0,858-1,4886,46,242,Blue,Light Gray
1,1591-1,11940,19,40,White,Blue
2,1592-1,14106,133,495,Black,Yellow


## export to csv

In [24]:
save_path = r'data/brickset.csv'
brickset.to_csv(save_path, index=False)
f'{round(os.path.getsize(save_path) /1e6, 2)} mb'

'1.33 mb'